(dask-dataframe-shuffle)=
# Shuffle

在分布式场景下，`sort`，`merge`，`groupby` 有可能会在不同 Worker 之间交换数据，即 Shuffle。这些 pandas 算子在单机上实现起来比较简单，但是在大数据分布式计算场景，实现起来并不简单。
Dask 在 `2023.1` 版本之后提供了一种新的 Shuffle 方法，可以加速大部分计算任务。

## `groupby`

{numref}`dataframe-groupby` 展示了 `groupby` 在单机上的操作流程，它主要有三个阶段：分组、聚合、输出。分布式场景下，不同的数据分布在不同的 Partition 下。

```{figure} ../img/ch-dask-dataframe/groupby.svg
---
width: 600px
name: dataframe-groupby
---
DataFrame groupby 示意图
```

* `groupby(indexed_columns).agg()` 和 `groupby(indexed_columns).apply(user_def_fn)` 性能最好。`indexed_columns` 指的是索引列 Key，`agg` 指的是 Dask DataFrame 提供的官方的 `sum`，`mean`，`nunique` 等聚合方法。因为 `indexed_columns` 是排过序的了，可以很快地对 `indexed_columns` 进行分组，Shuffle 数据量不大。
* `groupby(non_indexed_columns).agg()` 的数据交换量要更大一些，`agg` 是 Dask 官方提供的方法，做过一些优化。
* `groupby(non_indexed_columns).apply(user_def_fn)` 的成本最高。它既要对所有数据进行交换，又要执行用户自定义的函数，
